## Setup

In [1]:
!git clone https://github.com/neural-maze/agentic_patterns

fatal: destination path 'agentic_patterns' already exists and is not an empty directory.


In [2]:
%cd agentic_patterns

/content/agentic_patterns


In [3]:
!pip install groq==0.10.0 jupyter==1.0.0 python-dotenv==1.0.1 colorama==0.4.6 types-colorama==0.4.15.20240311 graphviz==0.20.3 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
agentic-patterns 1.0.0 requires groq<0.10.0,>=0.9.0, but you have groq 0.10.0 which is incompatible.


# Planning Pattern - ReAct Technique

<img src="https://github.com/neural-maze/agentic_patterns/blob/main/img/planning_pattern.png?raw=1" alt="Alt text" width="500"/>

---

So, we've seen agents capable of reflecting and using tools to access the outside world. But ... **what about planning**,
i.e. deciding what sequence of steps to follow to accomplish a large task?

That is exactly what the Planning Pattern provides; ways for the LLM to break a task into **smaller, more easily accomplished subgoals** without losing track of the end goal.

The most paradigmatic example of the planning pattern is the [**ReAct**](https://react-lm.github.io/) technique, displayed in the diagram above.

In this notebook, you'll learn how this technique actually works. This is the **third lesson** of the "Agentic Patterns from Scratch" series. Take a look
at the previous lessons if you haven't!

* [First Lesson: The Reflection Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/reflection_pattern.ipynb)
* [Second Lesson: The Tool Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/tool_pattern.ipynb)

## Relevant imports and Groq Client

We start by importing all the libraries we'll be using in this tutorial as well as the Groq client.

In [14]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq(api_key="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")

> If you are not familiar with the `tool` decorator, changes are you are missed the previous tutorial about the Tool Pattern. Check the video [here](https://www.youtube.com/watch?v=ApoDzZP8_ck&t=671s&ab_channel=TheNeuralMaze).

## A System Prompt for the ReAct Loop

As we did with the Tool Pattern, we also need a System Prompt for the ReAct technique. This System Prompt is very similar, the difference is that it describes the ReAct loop, so that the LLM is aware of
the three operations it's allowed to use:

1. Thought: The LLM will think about which action to take
2. Action: The LLM will use a Tool to "act on the environment"
3. Observation: The LLM will observe the tool output and reflect on the next thing to do.

Another key difference from the Tool Pattern System Prompt is that we are going to enclose all the messages with tags, like these: <thought></thought>, <observation></observation>. We could implement the ReAct logic without these tags, but I found it eeasier for the LLM to understand the instructions this way.

Ok! So without further ado, there's the prompt!

In [5]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools>
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

## Example step by step

### Defining the Tools

Let's build an example that involves the use of three tools, like the following ones.

In [6]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."

    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

Remember that the `@tool` operator allows us to convert a Python function into a `Tool` automatically. We cana check that very easily with some of the functions above.

In [7]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


### Adding the Tools signature to the System Prompt

Now, we just concatenate the tools signature and add them to the System Prompt.

In [8]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [9]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"prop

In [10]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [11]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

### ReAct Loop Step 1

In [12]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]


In [15]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result. I'll start by summing 1234 and 5678.</thought>
<tool_call>{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>


In [16]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 2

In [17]:
tool_call = extract_tag_content(output, tag="tool_call")

In [18]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}'], found=True)

In [19]:
tool_call = json.loads(tool_call.content[0])

In [20]:
tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [21]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [22]:
assert tool_result == 1234 + 5678

In [23]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 3

In [24]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.</thought>
<tool_call>{"name": "multiply_two_elements","arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [25]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 4

In [26]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [27]:
tool_result

34560

In [28]:
assert tool_result == (1234 + 5678) * 5

In [29]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 5

In [30]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the product of 6912 and 5, which is 34560. Now, I need to compute the logarithm of this result.</thought>
<tool_call>{"name": "compute_log","arguments": {"x": 34560}, "id": 2}</tool_call>


In [31]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 6

In [32]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [33]:
tool_result

10.450452222917992

In [34]:
assert tool_result == math.log((1234 + 5678) * 5)

In [35]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 7

In [36]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the logarithm of 34560, which is approximately 10.45. I can now provide the final answer.</thought>
<response>The sum of 1234 and 5678 is 6912. Multiplying this result by 5 gives 34560. The logarithm of this result is approximately 10.45.</response>


## Putting it all together

In [39]:
import json
import re

from colorama import Fore
from dotenv import load_dotenv
from groq import Groq

from agentic_patterns.tool_pattern.tool import Tool
from agentic_patterns.tool_pattern.tool import validate_arguments
from agentic_patterns.utils.completions import build_prompt_structure
from agentic_patterns.utils.completions import ChatHistory
from agentic_patterns.utils.completions import completions_create
from agentic_patterns.utils.completions import update_chat_history
from agentic_patterns.utils.extraction import extract_tag_content

load_dotenv()

BASE_SYSTEM_PROMPT = ""


REACT_SYSTEM_PROMPT = """
You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools>
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""


class ReactAgent:
    """
    A class that represents an agent using the ReAct logic that interacts with tools to process
    user inputs, make decisions, and execute tool calls. The agent can run interactive sessions,
    collect tool signatures, and process multiple tool calls in a given round of interaction.

    Attributes:
        client (Groq): The Groq client used to handle model-based completions.
        model (str): The name of the model used for generating responses. Default is "llama-3.3-70b-versatile".
        tools (list[Tool]): A list of Tool instances available for execution.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool instances.
    """

    def __init__(
        self,
        tools: Tool | list[Tool],
        model: str = "llama-3.3-70b-versatile",
        system_prompt: str = BASE_SYSTEM_PROMPT,
    ) -> None:
        self.client = Groq(api_key="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")
        self.model = model
        self.system_prompt = system_prompt
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    def run(
        self,
        user_msg: str,
        max_rounds: int = 10,
    ) -> str:
        """
        Executes a user interaction session, where the agent processes user input, generates responses,
        handles tool calls, and updates chat history until a final response is ready or the maximum
        number of rounds is reached.

        Args:
            user_msg (str): The user's input message to start the interaction.
            max_rounds (int, optional): Maximum number of interaction rounds the agent should perform. Default is 10.

        Returns:
            str: The final response generated by the agent after processing user input and any tool calls.
        """
        user_prompt = build_prompt_structure(
            prompt=user_msg, role="user", tag="question"
        )
        if self.tools:
            self.system_prompt += (
                "\n" + REACT_SYSTEM_PROMPT % self.add_tool_signatures()
            )

        chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=self.system_prompt,
                    role="system",
                ),
                user_prompt,
            ]
        )

        if self.tools:
            # Run the ReAct loop for max_rounds
            for _ in range(max_rounds):

                completion = completions_create(self.client, chat_history, self.model)

                response = extract_tag_content(str(completion), "response")
                if response.found:
                    return response.content[0]

                thought = extract_tag_content(str(completion), "thought")
                tool_calls = extract_tag_content(str(completion), "tool_call")

                update_chat_history(chat_history, completion, "assistant")

                print(Fore.MAGENTA + f"\nThought: {thought.content[0]}")

                if tool_calls.found:
                    observations = self.process_tool_calls(tool_calls.content)
                    print(Fore.BLUE + f"\nObservations: {observations}")
                    update_chat_history(chat_history, f"{observations}", "user")

        return completions_create(self.client, chat_history, self.model)


In [40]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])

In [41]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")


Thought: I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result. I will first call the sum_two_elements function to get the sum.

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}

Thought: I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}

Thought: I have the product of 6912 and 5, which is 34560. Now, I need to compute the logarithm of this result.

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 2}

Tool result: 
10.450452222917992

Observations: {2: 10.450452222917992}


'The sum of 1234 and 5678 is 6912. Multiplying this result by 5 gives 34560. The logarithm of this result is approximately 10.45.'

---

We did it!! A ReAct Agent working as expected, completely from Scratch! 🚀🚀🚀🚀